# Build a deep CNN model with adversarial loss for CIFAR-10

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
from keras import backend as K
from keras.backend import gradients
from keras.layers import Input, Conv2D, Lambda, Flatten,Activation, Dense,Dropout, MaxPooling2D, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras.optimizers import Adam, rmsprop
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [3]:
import keras
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

ntrain = 50000
ntest = 10000

x_train = x_train.astype('float32')[:ntrain]
x_test = x_test.astype('float32')[:ntest]
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, 10)[:ntrain]
y_test = keras.utils.to_categorical(y_test, 10)[:ntest]

# https://www.cs.toronto.edu/~kriz/cifar.html
label_map = {0:'Airplane', 1:'Automobile', 2:'Bird', 3:'Cat', 4:'Deer', 5:'Dog', 6:'Frog', 7:'Horse', 8:'Ship', 9:'Truck'}
label_list = ['Airplane','Automobile','Bird','Cat','Deer','Dog','Frog','Horse','Ship','Truck']

In [4]:
x_train[0].shape

(32, 32, 3)

In [5]:
import matplotlib.pyplot as plt
plt.imshow(x_train[0])

In [6]:
def build_model(num_classes):
    image_input = Input(shape=(32, 32, 3))
    x = Conv2D(32, (3,3), padding='same')(image_input)
    x = Activation('relu')(x)
    x = Conv2D(32, (3,3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(64, (3,3), padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3,3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes)(x)
    y = Activation('softmax')(x)
    
    return Model(inputs=image_input, outputs=y)

In [15]:
# Adversarial loss function, tensorboard visualization callback

# Keras callback: https://github.com/keras-team/keras/blob/master/keras/callbacks.py#L942
# start tensorboard: tensorboard --logdir=/home/sunil_khanal/tf-logs --samples_per_plugin=images=100
# view tensorboard: http://localhost:6006/#images

from time import time
from keras.callbacks import Callback, TensorBoard


import tensorflow as tf
import numpy as np

epsilon = 0.05
alpha = 0.5

def compute_gradient(tensor, var):
    gradients = tf.gradients(tensor, var)
    gradients = [gradient if gradient is not None else tf.zeros_like(var) for gradient in gradients]
    return gradients

class AdversarialCallback(Callback):
    """Callback to compute adversarially perturbed image and corresponding prediction"""
    def __init__(self, log_dir, model):
        super(AdversarialCallback, self).__init__()
        self.log_dir = log_dir
        self.file_writer = tf.summary.FileWriter(log_dir + "/scalars")
        self.model = model
    
    def set_model(self, model):
        print("setting up model")
        self.model = model
        self.sess = K.get_session()
        
        self.x = tf.placeholder(tf.float32, shape=(1, 32, 32, 3), name="input_placeholder")
        tf.summary.image('original_image', self.x, max_outputs=10)
        
        self.y_true = tf.placeholder(tf.float32, shape=(1, 10))
        label_index = tf.argmax(self.y_true, axis=1)[0]
        tf.summary.scalar("true_label_index", label_index)
        
        self.true_label = tf.placeholder(tf.string, name="true_label_placeholder")
        
        # label list to lookup with tensor index
        label_list_tf = tf.convert_to_tensor(label_list, dtype=tf.string)
               
        y_pred = self.model(self.x)
        pred_label_index = tf.argmax(y_pred, axis=1)[0]
        pred_label = label_list_tf[pred_label_index]
        
        loss = K.categorical_crossentropy(self.y_true, y_pred)
        gradients = compute_gradient(loss, self.x)
        gradient = gradients[0]
        tf.summary.image('image_gradient_sign', tf.sign(gradient))
        
        perturbed_image = self.x + K.variable(value=epsilon) * tf.sign(gradient) ## use sign of gradient
        tf.summary.image('perturbed_image', perturbed_image)
        
        adv_pred = self.model(perturbed_image)
        adv_label_index = tf.argmax(adv_pred, axis=1)[0]
        adv_label = label_list_tf[adv_label_index]
        
        labels = tf.strings.join([self.true_label, pred_label, adv_label], ",")
        tf.summary.text("true_label-pred_label-adversarial_label", labels)
        
        self.merged = tf.summary.merge_all()
    
    def on_epoch_end(self, epoch, logs=None):
        # reach min performance before logging metrics
        if epoch < 20:
            return
        
        val_data = self.validation_data
        #print(len(val_data)) # 4 elements
        #print(val_data[0].shape) # x
        #print(val_data[1].shape) # y
        #print(val_data[2].shape)
        #print(val_data[3])
        
        input_image = val_data[0][epoch]
        input_image = np.array([input_image], dtype=float)
        
        # feed true labels and pred labels
        true_label = val_data[1][epoch]
        true_label_index = np.argmax(true_label)
        true_label_string = label_map[true_label_index]
        
        feed_dict = {self.x: input_image, self.y_true: true_label.reshape((1,10)), self.true_label: true_label_string}
        ret = self.sess.run([self.merged], feed_dict=feed_dict)
        self.file_writer.add_summary(ret[0], epoch)

def adversarial_loss(input_layer, model, tensorboard_writer=None):
    def loss(y_true, y_pred):
        return K.categorical_crossentropy(y_true, y_pred)
    
    def adv_loss(y_true, y_pred):
        reg_loss = loss(y_true, y_pred)
        
        # compute gradients and assign the first one
        gradients = K.gradients(reg_loss, input_layer)
        assert(len(gradients) == 1)
        gradient = gradients[0]
        
        # perturb image with the gradient
        image_perturb = input_layer + K.variable(value=epsilon) * tf.sign(gradient)
        
        # compute model output on the perturbed image and the loss
        adv_output = model(image_perturb)
        adv_gradient_loss = loss(y_true, adv_output)
        
        # combine and return losses, using alpha=0.5
        return alpha*reg_loss + (1 - alpha)*adv_gradient_loss 
    
    return adv_loss

In [ ]:
# Train with regular loss function
K.clear_session()
log_dir = "/home/sunil_khanal/tf-logs/{}".format(time())

model = build_model(10)
model.compile(rmsprop(lr=0.0001, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
callback = AdversarialCallback(log_dir, model)
model.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_test, y_test), verbose=1, callbacks=[callback])
model.save("cifar-10-regular-loss.h5")

In [19]:
# Train with adversarial loss function
K.clear_session()
log_dir = "/home/sunil_khanal/tf-logs/{}".format(time())

model = build_model(10)
model.compile(rmsprop(lr=0.0001, decay=1e-6), loss=adversarial_loss(model.inputs[0], model), metrics=['accuracy'])
callback = AdversarialCallback(log_dir, model)
model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_test, y_test), verbose=1, callbacks=[callback])
model.save("cifar-10-adversarial-loss.h5")

Train on 50000 samples, validate on 10000 samples
setting up model
Epoch 1/50
50000/50000 [==============================] - 31s 626us/step - loss: 2.0249 - acc: 0.3263 - val_loss: 1.9820 - val_acc: 0.4183
Epoch 2/50
50000/50000 [==============================] - 30s 593us/step - loss: 1.8425 - acc: 0.4348 - val_loss: 1.8460 - val_acc: 0.4891
Epoch 3/50
50000/50000 [==============================] - 30s 598us/step - loss: 1.7645 - acc: 0.4819 - val_loss: 1.8305 - val_acc: 0.5051
Epoch 4/50
50000/50000 [==============================] - 30s 598us/step - loss: 1.7134 - acc: 0.5125 - val_loss: 1.8137 - val_acc: 0.5288
Epoch 5/50
50000/50000 [==============================] - 30s 599us/step - loss: 1.6690 - acc: 0.5362 - val_loss: 1.7294 - val_acc: 0.5619
Epoch 6/50
50000/50000 [==============================] - 30s 595us/step - loss: 1.6351 - acc: 0.5559 - val_loss: 1.7186 - val_acc: 0.5720
Epoch 7/50
50000/50000 [==============================] - 30s 594us/step - loss: 1.6064 - acc: 0.56

# Load original and perturbed images

In [66]:
# Load the perturbed images and predict using the adversarially trained classifier
from PIL import Image
import os

img_folder = "/home/sunil_khanal/development/af-phishing/webpage_classifier/python/adversarial/images_cmp"

def load_image(filename):
    image = Image.open(os.path.join(img_folder, filename))
    return np.array([np.array(image)/255.])

dog_dog_frog_original = load_image("dog_dog_frog_image.png")
horse_horse_cat_original = load_image("horse_horse_cat_image.png")
ship_ship_truck_original = load_image("ship_ship_truck_image.png")
truck_truck_automobile_original = load_image("truck_truck_automobile_image.png")
dog_dog_cat_original = load_image("dog_dog_cat_image.png")

dog_dog_frog_perturbed = load_image("dog_dog_frog_perturbed.png")
horse_horse_cat_perturbed = load_image("horse_horse_cat_perturbed.png")
ship_ship_truck_perturbed = load_image("ship_ship_truck_perturbed.png")
truck_truck_automobile_perturbed = load_image("truck_truck_automobile_perturbed.png")
dog_dog_cat_perturbed = load_image("dog_dog_cat_perturbed.png")

# Load regular and adversarially trained models

In [67]:
from keras.models import load_model
from keras.utils.generic_utils import get_custom_objects

# This is not adversarial loss, used as placeholder for loading model
def adv_loss(y_true, y_pred):
    return K.categorical_crossentropy(y_true, y_pred)
get_custom_objects().update({"adv_loss": adv_loss})

def predict(model, data):
    preds = model.predict(data)
    index = np.argmax(preds)
    print(label_map[index])

adversarial_model = load_model("/home/sunil_khanal/development/af-phishing/webpage_classifier/python/adversarial/cifar-10-adversarial-loss.h5")
regular_model = load_model("/home/sunil_khanal/development/af-phishing/webpage_classifier/python/adversarial/cifar-10-regular-loss.h5")

# Predict original images using regular model

In [68]:
predict(regular_model, dog_dog_frog_original)
predict(regular_model, horse_horse_cat_original)
predict(regular_model, ship_ship_truck_original)
predict(regular_model, truck_truck_automobile_original)
predict(regular_model, dog_dog_cat_original)

Dog
Horse
Ship
Truck
Dog


# Predict perturbed images using regular model

In [69]:
predict(regular_model, dog_dog_frog_perturbed)
predict(regular_model, horse_horse_cat_perturbed)
predict(regular_model, ship_ship_truck_perturbed)
predict(regular_model, truck_truck_automobile_perturbed)
predict(regular_model, dog_dog_cat_perturbed)

Deer
Cat
Ship
Automobile
Cat


# Predict perturbed images using adversarial model

In [70]:
predict(adversarial_model, dog_dog_frog_perturbed)
predict(adversarial_model, horse_horse_cat_perturbed)
predict(adversarial_model, ship_ship_truck_perturbed)
predict(adversarial_model, truck_truck_automobile_perturbed)
predict(adversarial_model, dog_dog_cat_perturbed)

Dog
Horse
Ship
Truck
Dog
